In [42]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import re
from typing import List, Tuple, Dict, Optional
from zipfile import ZipFile

In [10]:
# Извлечение файлов из зип архива
with ZipFile("CROHME23.zip", "r") as myzip:
    myzip.extractall()

In [33]:
def parse_inkml(file_path: str) -> Dict:
    """
    Парсит InkML файл, извлекая LaTeX, трассировки и группировку символов

    Args:
        file_path: путь к InkML файлу

    Returns:
        Словарь с ключами:
        - 'latex': LaTeX-разметка формулы
        - 'traces': словарь {trace_id: список точек}
        - 'symbols': список информации о символах
        - 'mathml': MathML-разметка (если есть)
    """
    try:
        # Парсим XML с учетом пространства имен
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Регистрируем пространства имен
        ns = {
            'inkml': 'http://www.w3.org/2003/InkML',
            'mathml': 'http://www.w3.org/1998/Math/MathML'
        }

        # Извлекаем LaTeX
        latex = None
        for annotation in root.findall('inkml:annotation', ns):
            if annotation.get('type') == 'truth':
                latex = annotation.text.strip()
                break

        # Извлекаем MathML
        mathml = None
        for annotation in root.findall('inkml:annotationXML', ns):
            if annotation.get('type') == 'truth':
                mathml = ET.tostring(annotation, encoding='unicode')
                break

        # Извлекаем трассировки
        traces = {}
        for trace in root.findall('inkml:trace', ns):
            trace_id = trace.get('id')
            points = []

            # Парсим координаты
            for point_str in trace.text.strip().split(','):
                point_str = point_str.strip()
                if point_str:
                    coords = point_str.split()
                    if len(coords) >= 2:
                        x, y = float(coords[0]), float(coords[1])
                        points.append((x, y))

            traces[trace_id] = points

        # Извлекаем группировку символов
        symbols = []
        for trace_group in root.findall('.//inkml:traceGroup', ns):
            # Пропускаем корневую группу
            if trace_group.get('xml:id') == '11':
                continue

            # Извлекаем символ
            symbol = None
            for annotation in trace_group.findall('inkml:annotation', ns):
                if annotation.get('type') == 'truth':
                    symbol = annotation.text.strip()
                    break

            # Извлекаем ссылки на трассировки
            trace_refs = []
            for trace_view in trace_group.findall('inkml:traceView', ns):
                trace_ref = trace_view.get('traceDataRef')
                if trace_ref:
                    trace_refs.append(trace_ref)

            # Извлекаем ссылку на MathML
            mathml_ref = None
            for annotation_xml in trace_group.findall('inkml:annotationXML', ns):
                mathml_ref = annotation_xml.get('href')
                break

            if symbol:
                symbols.append({
                    'symbol': symbol,
                    'trace_ids': trace_refs,
                    'mathml_ref': mathml_ref
                })

        return {
            'latex': latex,
            'traces': traces,
            'symbols': symbols,
            'mathml': mathml
        }

    except Exception as e:
        print(f"Ошибка при парсинге {file_path}: {str(e)}")
        return {
            'latex': None,
            'traces': {},
            'symbols': [],
            'mathml': None
        }

In [41]:
def normalize_coordinates(traces: Dict[str, List[Tuple[float, float]]],
                        img_size: Tuple[int, int] = (512, 256)) -> Dict[str, List[Tuple[float, float]]]:
    """
    Нормализует координаты трассировок к заданному размеру изображения

    Args:
        traces: словарь с трассировками
        img_size: целевой размер изображения (ширина, высота)

    Returns:
        Словарь с нормализованными трассировками
    """
    if not traces:
        return {}

    # Находим границы всех координат
    all_points = [point for trace in traces.values() for point in trace]
    if not all_points:
        return traces

    min_x = min(p[0] for p in all_points)
    max_x = max(p[0] for p in all_points)
    min_y = min(p[1] for p in all_points)
    max_y = max(p[1] for p in all_points)

    # Добавляем отступы
    margin = 0.05  # 5% отступ
    width = max_x - min_x
    height = max_y - min_y

    if width == 0:
        width = 1
    if height == 0:
        height = 1

    # Масштабируем координаты
    normalized_traces = {}
    for trace_id, points in traces.items():
        normalized_points = []
        for x, y in points:
            # Нормализация к [0, 1]
            norm_x = (x - min_x) / width
            norm_y = (y - min_y) / height

            # Применяем отступы
            norm_x = norm_x * (1 - 2 * margin) + margin
            norm_y = norm_y * (1 - 2 * margin) + margin

            # Масштабируем к размеру изображения
            scaled_x = norm_x * img_size[0]
            scaled_y = norm_y * img_size[1]

            normalized_points.append((scaled_x, scaled_y))

        normalized_traces[trace_id] = normalized_points

    return normalized_traces

In [46]:
def render_formula(traces: Dict[str, List[Tuple[float, float]]],
                  img_size: Tuple[int, int] = (512, 256),
                  line_width: int = 2) -> Optional['Image.Image']:
    """
    Рендерит формулу из трассировок в изображение

    Args:
        traces: словарь с трассировками
        img_size: размер изображения
        line_width: толщина линии

    Returns:
        PIL.Image или None в случае ошибки
    """
    try:
        from PIL import Image, ImageDraw
        import numpy as np

        # Нормализуем координаты
        normalized_traces = normalize_coordinates(traces, img_size)

        # Создаем изображение
        img = Image.new('RGB', img_size, color='white')
        draw = ImageDraw.Draw(img)

        # Рисуем трассировки
        for trace_id, points in normalized_traces.items():
            if len(points) > 1:
                # Преобразуем точки в формат для PIL
                pil_points = [(p[0], p[1]) for p in points]
                draw.line(pil_points, fill='black', width=line_width)

        return img

    except ImportError:
        print("Для рендеринга нужна библиотека PIL")
        return None
    except Exception as e:
        print(f"Ошибка при рендеринге: {str(e)}")
        return None

In [ ]:
# Пример использования
if __name__ == "__main__":
    file_path = "TC11_CROHME23/INKML/train/CROHME2019/001-equation000.inkml"
    data = parse_inkml(file_path)

    print("=== LaTeX ===")
    print(data['latex'])

    print("\n=== Трассировки ===")
    print(f"Всего трассировок: {len(data['traces'])}")
    for trace_id, points in list(data['traces'].items())[:3]:  # Показываем первые 3
        print(f"Трассировка {trace_id}: {len(points)} точек")

    print("\n=== Символы ===")
    for symbol in data['symbols']:
        print(f"Символ: {symbol['symbol']}")
        print(f"  Трассировки: {symbol['trace_ids']}")
        print(f"  Ссылка на MathML: {symbol['mathml_ref']}")
    # Рендеринг изображения
    img = render_formula(data['traces'])
    if img:
        img.save("formula.png")
        print("\nИзображение сохранено как formula.png")

In [29]:
def parse_lg_file(file_path: str) -> Dict[str, List[Dict[str, Union[str, float]]]]:
    """
    Парсит .lg файлы SymLG формата

    Args:
        file_path: путь к файлу .lg

    Returns:
        Словарь с двумя ключами:
        - 'objects': список словарей с информацией о символах
        - 'relations': список словарей с отношениями между символами
    """
    objects = []
    relations = []

    with open(file_path, 'r', encoding='utf-8') as f:
        current_section = None

        for line in f:
            line = line.strip()

            # Пропускаем пустые строки и комментарии
            if not line or line.startswith('#'):
                # Определяем текущую секцию
                if line.startswith('# Objects('):
                    current_section = 'objects'
                elif line.startswith('# Relations from SRT:'):
                    current_section = 'relations'
                continue

            # Парсим в зависимости от секции
            if current_section == 'objects':
                obj = parse_object_line(line)
                if obj:
                    objects.append(obj)

            elif current_section == 'relations':
                rel = parse_relation_line(line)
                if rel:
                    relations.append(rel)

    return {
        'objects': objects,
        'relations': relations
    }

def parse_object_line(line: str) -> Dict[str, Union[str, float]]:
    """
    Парсит строку с описанием объекта (символа)

    Формат: O, <id>, <class>, <confidence>, <stroke_info>
    """
    parts = [part.strip() for part in line.split(',')]

    if len(parts) < 5 or parts[0] != 'O':
        return None

    try:
        return {
            'id': parts[1],
            'class': parts[2],
            'confidence': float(parts[3]),
            'stroke_info': parts[4]
        }
    except (ValueError, IndexError):
        return None

def parse_relation_line(line: str) -> Dict[str, Union[str, float]]:
    """
    Парсит строку с описанием отношения между символами

    Формат: R, <id1>, <id2>, <relation_type>, <confidence>
    """
    parts = [part.strip() for part in line.split(',')]

    if len(parts) < 5 or parts[0] != 'R':
        return None

    try:
        return {
            'id1': parts[1],
            'id2': parts[2],
            'relation_type': parts[3],
            'confidence': float(parts[4])
        }
    except (ValueError, IndexError):
        return None

In [ ]:
# Пример использования
if __name__ == "__main__":
    file_path = "TC11_CROHME23/SymLG/train/CROHME2019_train/001-equation000.lg"
    data = parse_lg_file(file_path)

    print("=== Objects ===")
    for obj in data['objects']:
        print(f"{obj['id']}: {obj['class']} (confidence: {obj['confidence']})")

    print("\n=== Relations ===")
    for rel in data['relations']:
        print(f"{rel['id1']} --{rel['relation_type']}--> {rel['id2']} (confidence: {rel['confidence']})")

In [ ]:
import cv2
import matplotlib.pyplot as plt

def visualize_boxes(image_path, sym_data):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    for _, row in sym_data.iterrows():
        x1, y1 = int(row['left']), int(row['top'])
        x2, y2 = int(row['right']), int(row['bottom'])

        # Рисуем bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), (255, 0, 0), 2)

        # Добавляем метку класса
        cv2.putText(img, row['class'], (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255,0,0), 2)

    plt.figure(figsize=(12, 8))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

# Пример использования
visualize_boxes("/TC11_CROHME23/IMG/train/CROHME2019/001-equation000.png", sym_data)